**Edit:** Quadratic Kappa Metric es igual a cohen kappa metric en Sci-kit learn @ sklearn.metrics.cohen_kappa_score cuando los pesos están configurados en 'Quadratic'. Gracias a Johannes por averiguarlo.

## Que es Quadratic Weighted Kappa?  

Las presentaciones se califican en función de quadratic weighted kappa (**kappa ponderada cuadrática en español**), que mide el acuerdo entre dos calificaciones. Esta métrica generalmente varía de 0 (acuerdo aleatorio entre evaluadores) a 1 (acuerdo completo entre evaluadores). En el caso de que haya menos acuerdo entre los evaluadores que lo esperado por casualidad, la métrica puede ir por debajo de 0. El quadratic weighted kappa  se calcula entre los puntajes esperados / conocidos y los puntajes pronosticados. <br>


Los resultados tienen 5 clasificaciones posibles, 0,1,2,3,4. La quadratic weighted kappa se calcula de la siguiente manera. Primero, se construye una matriz O del histograma N x N, de manera que Oi, j corresponde al número de registros de adopción que tienen una calificación de **i** (real) y recibieron una calificación **j** predicha. Una matriz de ponderaciones N-por-N, w, se calcula en función de la diferencia entre los puntajes de calificación reales y pronosticados.

Se calcula una matriz de histogramas N-por-N de las calificaciones esperadas, E, suponiendo que no hay correlación entre las puntuaciones de las calificaciones. Esto se calcula como el producto externo entre el vector de histogramas de las clasificaciones reales y el vector de histogramas de las clasificaciones pronosticadas, normalizado de tal manera que E y O tienen la misma suma.

A partir de estas tres matrices, se calcula quadratic weighted kappa.

### Partiendo la fórmula en partes

#### Desglose de 5 pasos para Weighted Kappa Metric

- Primero, cree una matriz de confusión de clase múltiple `O` entre las calificaciones predichas y las reales.
- Segundo, construya una matriz de ponderación `w` que calcule la ponderación entre las calificaciones reales y las pronosticadas.
- Tercero, calcule `value_counts ()` para cada calificación entre los valores pronosticados y en reales.
- Cuarto, calcule `E`, que es el producto externo de dos vectores value_count
- Quinto, normalizar la matriz `E` y` O`
- Caclulate, kappa ponderado según la fórmula

#### Cada paso explicado

**Paso-1:** En el Paso-1, estaremos calculando una 'confusión_matriz' entre los valores Predicho y Real. <a href="https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/">Este</a> es un gran recurso para saber más sobre `confusion_matrix`. <br>
**Paso 2:** En el Paso 2,  cada elemento está ponderado. Las predicciones que están más alejadas de los datos reales están marcadas con dureza que las predicciones que están más cerca de los datos reales. Tendremos una puntuación menor si nuestra predicción es 5 y la real es 3 en comparación con una predicción de 4 en el mismo caso. <br>
**Paso 3:** Creamos dos vectores, uno para las predicciones y otro para datos reales, que nos dice cuántos valores de cada calificación existen en ambos vectores. <br>
**Paso 4:** `E` es la Matriz Esperada que es el producto externo de los dos vectores calculados en el paso 3. <br>
**Paso 5:** Normalizar ambas matrices para tener la misma suma. Como es más fácil que la suma sea '1', simplemente dividiremos cada matriz por su suma para normalizar los datos. <br>
**Paso 6:** Numerador y denominador calculados de Kappa ponderados y devuelven la Weighted Kappa Metric como `1-(num/den)`

### Intepretando la Quadratic Weighted Kappa Metric 

Un Kappa ponderado es una métrica que se utiliza para calcular la cantidad de similitud entre las predicciones y los datos reales. Se otorga una puntuación perfecta de `1.0` cuando las predicciones y los datos reales son iguales. <br>
Considerando que, la puntuación menos posible es `-1` que se da cuando las predicciones están más lejos de los reales. En nuestro caso, consideremos que todos los datos reales fueron 0 y todas las predicciones fueron 4. Esto llevaría a una puntuación de `QWKP` de` -1`. <br>
El objetivo es acercarse lo más posible a 1. En general, una puntuación de 0,6+ se considera una puntuación realmente buena.

## Creando nuestro propio Quadratic Weighted Kappa Metric

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

Para el propósito de la explicación, supondremos que los vectores reales y predichos son los siguientes.

In [ ]:
actuals = np.array([4, 4, 3, 4, 4, 4, 1, 1, 2, 1])
preds   = np.array([0, 2, 1, 0, 0, 0, 1, 1, 2, 1])

In [ ]:
actuals.shape,preds.shape

### Paso 1: Matrix de confusion

In [ ]:
O = confusion_matrix(actuals, preds); O

In [ ]:
#confusion_matrix(actuals, preds)

### Paso 2: Matriz ponderada

Una matriz de pesos N-por-N, w, se calcula en función de la diferencia entre los puntajes de calificación reales y pronosticados.

In [ ]:
w = np.zeros((5,5)); w

In [ ]:
for i in range(len(w)):
    for j in range(len(w)):
        w[i][j] = float(((i-j)**2)/16) #Según la fórmula, para esta competición, N=5

In [ ]:
w

Tenga en cuenta que todos los valores que se encuentran en la diagonal son los menos penalizados con una penalización de 0, mientras que las predicciones y los datos reales más alejados entre sí son los más penalizados.

### Paso 3: Histograma

In [ ]:
N=5
act_hist=np.zeros([N])
for item in actuals: 
    act_hist[item]+=1
    
pred_hist=np.zeros([N])
for item in preds: 
    pred_hist[item]+=1

In [ ]:
print(f'Recuento de valores reales:{act_hist}, Recuento de valores predichos:{pred_hist}')

Por lo tanto, tenemos en los datos reales:<br>
3 valores con calificación de adopción 1<br>
1 valor con calificación de adopción 2<br>
1 valor con calificación de adopción 3<br>
5 valores con calificación de adopción de 4

### Paso 4: Valor esperado (producto externo de los histogramas)

La matriz esperada se calcula como el producto externo entre el vector de histogramas de las clasificaciones reales y el vector de histogramas de las clasificaciones predichas

In [ ]:
E = np.outer(act_hist, pred_hist); E

### Paso 5: Normalizar la matriz E y O

`E` y `O` están normalizados de modo que E y O tienen la misma suma.

In [ ]:
E = E/E.sum(); E.sum()

In [ ]:
O = O/O.sum(); O.sum()

In [ ]:
E

In [ ]:
O

### Paso 6: Calcular Kappa ponderada

In [ ]:
num=0
den=0
for i in range(len(w)):
    for j in range(len(w)):
        num+=w[i][j]*O[i][j]
        den+=w[i][j]*E[i][j]
 
weighted_kappa = (1 - (num/den)); weighted_kappa

### Compare el resultado con la métrica existente

El siguiente código para calcular la métrica de Kappa ponderada fue utilizado por Abhishek en su núcleo https://www.kaggle.com/abhishek/maybe-something-interesting-here. 

In [ ]:
# The following 3 functions have been taken from Ben Hamner's github repository
# https://github.com/benhamner/Metrics
def Cmatrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = Cmatrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

In [ ]:
quadratic_weighted_kappa(actuals, preds)

Nuestro resultado coincide con la métrica kappa ponderada cuadrática existente.

## Reescribe la función métrica de Kappa cuadrática

In [ ]:
def quadratic_kappa(actuals, preds, N=5):
    """This function calculates the Quadratic Kappa Metric used for Evaluation in the PetFinder competition
    at Kaggle. It returns the Quadratic Weighted Kappa metric score between the actual and the predicted values 
    of adoption rating."""
    w = np.zeros((N,N))
    O = confusion_matrix(actuals, preds)
    for i in range(len(w)): 
        for j in range(len(w)):
            w[i][j] = float(((i-j)**2)/(N-1)**2)
    
    act_hist=np.zeros([N])
    for item in actuals: 
        act_hist[item]+=1
    
    pred_hist=np.zeros([N])
    for item in preds: 
        pred_hist[item]+=1
                         
    E = np.outer(act_hist, pred_hist);
    E = E/E.sum();
    O = O/O.sum();
    
    num=0
    den=0
    for i in range(len(w)):
        for j in range(len(w)):
            num+=w[i][j]*O[i][j]
            den+=w[i][j]*E[i][j]
    return (1 - (num/den))

In [ ]:
actuals

In [ ]:
preds

In [ ]:
quadratic_kappa(actuals, preds)

**¿Qué pasa si los datos reales y las predicciones coinciden con el 100%?**

In [ ]:
actuals = np.array([4, 4, 3, 4, 4, 4, 1, 1, 2, 0])
preds   = np.array([4, 4, 3, 4, 4, 4, 1, 1, 2, 0])
quadratic_kappa(actuals, preds)

Publicacion de paper con mas detalles 29 Sep 2015 21:30:43 UTC: https://arxiv.org/abs/1509.07107v2

Disclaimer: Este Kernel se basa en el original: https://www.kaggle.com/aroraaman/quadratic-kappa-metric-explained-in-5-simple-steps se realizo la traduccion y modificacion para fines academicos y no como copia